In [1]:
import os
import soundfile as sf
import time
import math
import numpy as np
from natsort import natsorted
import soundfile as sf
import librosa
import h5py
import tensorflow as tf
import os
from sklearn import preprocessing
from sklearn.utils import shuffle

In [2]:
file_series = str(1)
addr_X_file = 'TSP/Predicted/predicted_TSP_FFT'  + file_series + '.hdf5'
addr_y_file = 'TSP/Organized/concatenated/Second_set/Train2_TSP_clean_FFTs'  + file_series + '.hdf5'

In [3]:
a = h5py.File('TSP/Organized/concatenated/Second_set/all_len'  + file_series + '.hdf5','r')
all_len = a['all_len'][:]
print(all_len)
a.close()

[     0    186    387 ... 474040 474199 474382]


In [9]:
load_size = int(np.floor(all_len[-1] / 40))
index = np.arange(0, all_len[-1], load_size)

In [12]:
load_size

11859

In [6]:
def np_to_tfrecords(X, Y, file_path_prefix, verbose=True):
    """
    Converts a Numpy array (or two Numpy arrays) into a tfrecord file.
    For supervised learning, feed training inputs to X and training labels to Y.
    For unsupervised learning, only feed training inputs to X, and feed None to Y.
    The length of the first dimensions of X and Y should be the number of samples.
    
    Parameters
    ----------
    X : numpy.ndarray of rank 2
        Numpy array for training inputs. Its dtype should be float32, float64, or int64.
        If X has a higher rank, it should be rshape before fed to this function.
    Y : numpy.ndarray of rank 2 or None
        Numpy array for training labels. Its dtype should be float32, float64, or int64.
        None if there is no label array.
    file_path_prefix : str
        The path and name of the resulting tfrecord file to be generated, without '.tfrecords'
    verbose : bool
        If true, progress is reported.
    
    Raises
    ------
    ValueError
        If input type is not float (64 or 32) or int.
    
    """
    def _dtype_feature(ndarray):
        """match appropriate tf.train.Feature class with dtype of ndarray. """
        assert isinstance(ndarray, np.ndarray)
        dtype_ = ndarray.dtype
        if dtype_ == np.float64 or dtype_ == np.float32:
            return lambda array: tf.train.Feature(float_list=tf.train.FloatList(value=array))
        elif dtype_ == np.float64:
            return lambda array: tf.train.Feature(int64_list=tf.train.Int64List(value=array))
        else:  
            raise ValueError("The input should be numpy ndarray. \
                               Instaed got {}".format(ndarray.dtype))
            
    assert isinstance(X, np.ndarray)
    assert len(X.shape) == 2  # If X has a higher rank, 
                               # it should be rshape before fed to this function.
    assert isinstance(Y, np.ndarray) or Y is None
    
    # load appropriate tf.train.Feature class depending on dtype
    dtype_feature_x = _dtype_feature(X)
    if Y is not None:
        assert X.shape[0] == Y.shape[0]
        assert len(Y.shape) == 2
        dtype_feature_y = _dtype_feature(Y)            
    
    # Generate tfrecord writer
    result_tf_file = file_path_prefix + '.tfrecords'
    writer = tf.python_io.TFRecordWriter(result_tf_file)
    if verbose:
        print("Serializing {:d} examples into {}".format(X.shape[0], result_tf_file))
        
    # iterate over each sample,
    # and serialize it as ProtoBuf.
    for idx in range(X.shape[0]):
        x = X[idx]
        if Y is not None:
            y = Y[idx]
        
        d_feature = {}
        d_feature['X'] = dtype_feature_x(x)
        if Y is not None:
            d_feature['Y'] = dtype_feature_y(y)
            
        features = tf.train.Features(feature=d_feature)
        example = tf.train.Example(features=features)
        serialized = example.SerializeToString()
        writer.write(serialized)
    
    if verbose:
        print("Writing {} done!".format(result_tf_file))

        

In [38]:
def dg_jae(addr_X_file, addr_y_file, all_len, window_size, file_series, load_size):
    
    global I
    h5f = {}
    X_file = h5py.File(addr_X_file,'r')
    y_file = h5py.File(addr_y_file,'r')

    
    flag = np.asarray(all_len)
    print('flags_shape=', flag.shape)
    r = []
    r0 = []
    r1 = []
    flag_start = 0
    k = 0
    all_shape = 0
    for I in range(len(flag)-1):
        X = X_file['predicted_TSP_FFT' + file_series][flag[I]:flag[I+1],:]
        
        half_len = X.shape[1]//2
        temp0 = X[:,0:half_len]
        temp0 = preprocessing.normalize(temp0, norm='l2', axis=1, copy=True)
        temp0 = np.concatenate((temp0,np.repeat(np.reshape(temp0[-1,:],(1,temp0.shape[1])),[window_size],axis=0)), axis=0)
        temp0 = np.concatenate((np.repeat(np.reshape(temp0[0,:],(1,temp0.shape[1])),[window_size],axis=0),temp0), axis=0)

        for i in range(0,temp0.shape[0]-2*window_size):
            r0.append(np.ndarray.flatten(temp0[i:i+window_size*2+1,:]))
                
        temp1 = X[:,half_len:]
        temp1 = preprocessing.normalize(temp1, norm='l2', axis=1, copy=True)
        temp1 = np.concatenate((temp1,np.repeat(np.reshape(temp1[-1,:],(1,temp1.shape[1])),[window_size],axis=0)), axis=0)
        temp1 = np.concatenate((np.repeat(np.reshape(temp1[0,:],(1,temp1.shape[1])),[window_size],axis=0),temp1), axis=0)

        for i in range(0,temp1.shape[0]-2*window_size):
            r1.append(np.ndarray.flatten(temp1[i:i+window_size*2+1,:]))       
           
        
        
        if np.asarray(r0).shape[0] > load_size:
            r = np.concatenate((np.asarray(r0),np.asarray(r1)),axis=1)
            y = y_file['Train2_TSP_clean_FFTs' + file_series][:,flag_start:flag_start + len(r)]
            flag_start = r.shape[0] + 1
            np_to_tfrecords(np.asarray(r), y.T, 'TSP/Organized/concatenated/TFr/dataset_context' + str(k), verbose=True)
            print('r_shape=', r.shape)
            all_shape = r.shape[0] + all_shape
            r0 = []
            r1 = []
            k = k + 1
            print('all_shape=',all_shape)
   
    
    r = np.concatenate((np.asarray(r0),np.asarray(r1)),axis=1)
    y = y_file['Train2_TSP_clean_FFTs' + file_series][:,flag_start:flag_start + len(r)]
    flag_start = r.shape[0] + 1
    np_to_tfrecords(np.asarray(r), y.T, 'TSP/Organized/concatenated/TFr/dataset_context' + str(k), verbose=True)
    print('r_shape=', r.shape)
    all_shape = r.shape[0] + all_shape
    r0 = []
    r1 = []
    k = k + 1
        
    print('all_shape_full=', all_shape)
 

In [39]:
qqq = dg_jae(addr_X_file, addr_y_file, all_len, 2, str(1), load_size)

flags_shape= (2401,)
Serializing 11910 examples into TSP/Organized/concatenated/TFr/dataset_context0.tfrecords
Writing TSP/Organized/concatenated/TFr/dataset_context0.tfrecords done!
r_shape= (11910, 10250)
all_shape= 11910
Serializing 11913 examples into TSP/Organized/concatenated/TFr/dataset_context1.tfrecords
Writing TSP/Organized/concatenated/TFr/dataset_context1.tfrecords done!
r_shape= (11913, 10250)
all_shape= 23823
Serializing 11948 examples into TSP/Organized/concatenated/TFr/dataset_context2.tfrecords
Writing TSP/Organized/concatenated/TFr/dataset_context2.tfrecords done!
r_shape= (11948, 10250)
all_shape= 35771
Serializing 11925 examples into TSP/Organized/concatenated/TFr/dataset_context3.tfrecords
Writing TSP/Organized/concatenated/TFr/dataset_context3.tfrecords done!
r_shape= (11925, 10250)
all_shape= 47696
Serializing 11967 examples into TSP/Organized/concatenated/TFr/dataset_context4.tfrecords
Writing TSP/Organized/concatenated/TFr/dataset_context4.tfrecords done!
r_sha

In [53]:
qqq.shape

(186, 10250)

In [9]:
# k = 0
# for I in index:
#     if data_len-I<load_size:
#         X = X_file['Train_TSP_mixed_FFT' + file_series][:,I:]
#         y = y_file['Train_TSP_clean_FFTs' + file_series][:,I:]
#     else:
#         X = X_file['Train_TSP_mixed_FFT' + file_series][:,I:I+load_size]
#         y = y_file['Train_TSP_clean_FFTs' + file_series][:,I:I+load_size]
# #     Normalization
#     X = preprocessing.normalize(X, norm='l2', axis=0, copy=True)
# #     Mask
#     a = y[0:1025]
#     b = y[1025:]
#     Mask = a/(a+b)
#     Mask = np.concatenate((Mask,b/(a+b)), axis = 0)
#     np_to_tfrecords(X.T, Mask.T, 'TSP/Organized/concatenated/TFr/dataset' + str(k), verbose=True)
#     k = k+1

Serializing 95074 examples into TSP/Organized/concatenated/TFr/dataset0.tfrecords
Writing TSP/Organized/concatenated/TFr/dataset0.tfrecords done!
Serializing 95074 examples into TSP/Organized/concatenated/TFr/dataset1.tfrecords
Writing TSP/Organized/concatenated/TFr/dataset1.tfrecords done!
Serializing 95074 examples into TSP/Organized/concatenated/TFr/dataset2.tfrecords
Writing TSP/Organized/concatenated/TFr/dataset2.tfrecords done!
Serializing 95074 examples into TSP/Organized/concatenated/TFr/dataset3.tfrecords
Writing TSP/Organized/concatenated/TFr/dataset3.tfrecords done!
Serializing 95074 examples into TSP/Organized/concatenated/TFr/dataset4.tfrecords
Writing TSP/Organized/concatenated/TFr/dataset4.tfrecords done!
Serializing 1 examples into TSP/Organized/concatenated/TFr/dataset5.tfrecords
Writing TSP/Organized/concatenated/TFr/dataset5.tfrecords done!
